In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import os

# データフレーム読み込み
train_df = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")

# データ結合
all_df = pd.concat([train_df,test_df],axis=0,ignore_index=True)
max_row = len(all_df)

In [ ]:
# all_df.info() # 特徴量、欠損、型確認

In [ ]:
# test_df.head()

In [2]:
# 数値列とカテゴリ列を取得
num_col = []
cat_col = []

train_df = train_df.drop(["id","y"],axis=1)

for col in train_df.columns:
    if train_df[col].dtypes!="object":
        num_col.append(col)
    else:
        cat_col.append(col)

In [ ]:
# all_df.head(1)

In [3]:
# 訓練データとテストデータに分離
train = all_df[:len(train_df)]
test = all_df[len(train_df):]

# 訓練データをx,yに分割
X_train = train.drop(["id","y"],axis=1)
y_train = train["y"]
X_test = test.drop(["id","y"],axis=1)

In [4]:
# ラベルエンコード
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

for col in X_train.columns:
    if col in cat_col:
        
        # 訓練データでfit
        LE.fit(X_train[col])

        # 訓練・テストデータをそれぞれtransform
        X_train[col] = LE.transform(X_train[col])
        X_test[col] = LE.transform(X_test[col])

# データフレーム変換
X_train_df = pd.DataFrame(X_train,columns=cat_col)
X_test_df = pd.DataFrame(X_test,columns=cat_col)

### 【Light GBM】

In [ ]:
# ###################################################
# ############ Light GBM ############################
# ###################################################
# import lightgbm as lgb
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

# # 学習、バリデーションデータ
# pred_lgb = np.zeros(len(train_df))
# pred_lgb_test = np.zeros(len(test_df))
# models_lgb = []

# # 入力データ
# X = X_train
# y = y_train

# # 評価履歴を格納する辞書
# evals_result_lgb = {}

# # パラメータ
# lgbm_params = {
#     'objective': 'binary',
#     "device": "cpu",
#     # "device": "gpu",
#     'metric': 'auc',
#     'verbose': -1,              # ログ出力の制御
#     'boosting_type': 'gbdt',
#     # 'learning_rate': 0.01,
#     # 'feature_fraction': 0.6956717916553479,
#     # 'num_leaves':       153,
#     # 'bagging_fraction': 0.5279852787927486,
#     # 'bagging_freq':     4,
#     # 'lambda_l1':        0.004603414256652151,
#     # 'lambda_l2':        0.013858762846118894,
#     # 'min_data_in_leaf': 87,
#     }

# # クロスバリデーション
# # kf = KFold(n_splits=5, shuffle=True, random_state=42)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # for fold,(train_idx,valid_idx) in enumerate(kf.split(X)):
# for fold,(train_idx,valid_idx) in enumerate(skf.split(X,y)):

#     print("#"*25)
#     print(f"### Folf {fold+1}")
#     print("#"*25)
    
#     X_train_kf = X.iloc[train_idx,:]
#     y_train_kf = y.iloc[train_idx]
#     X_valid_kf = X.iloc[valid_idx,:]
#     y_valid_kf = y.iloc[valid_idx]
    
#     # データセット作成
#     lgb_train_lgb = lgb.Dataset(X_train_kf,y_train_kf)
#     lgb_valid_lgb = lgb.Dataset(X_valid_kf,y_valid_kf)


#     # 学習
#     model_lgb = lgb.train(
#         lgbm_params,
#         lgb_train_lgb,
#         num_boost_round=2000,
#         valid_sets=[lgb_train_lgb,lgb_valid_lgb],
#         valid_names=["train","valid"],
#         callbacks=[
#             lgb.early_stopping(stopping_rounds=100,verbose=False),
#             lgb.record_evaluation(evals_result_lgb),
#             lgb.log_evaluation(100),])

#     # 各foldでのバリデーション予測
#     pred_lgb[valid_idx] = model_lgb.predict(
#         X_valid_kf, num_iteration=model_lgb.best_iteration)
    
#     # モデルの追加
#     models_lgb.append(model_lgb)

#     # テストの予測
#     pred_lgb_test = pred_lgb_test + model_lgb.predict(
#         X_test, num_iteration=model_lgb.best_iteration)

# # FOLD数で割る
# pred_lgb_test = pred_lgb_test/5

In [ ]:
# # 訓練データのスコア
# from sklearn.metrics import f1_score
# import matplotlib.pyplot as plt

# AUC_lgb = roc_auc_score(y,pred_lgb)
# F1_lgb = f1_score(y,np.round(pred_lgb,0))
# print(f"LGB: AUC score = {AUC_lgb}, F1 = {F1_lgb}")

# # 学習曲線
# # lgb.plot_metric(evals_result_lgb,title="LightGBM AUC",)

In [ ]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_lgb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

### 【XGBoost】

In [ ]:
# #################################################
# ############ XGBoost ############################
# #################################################
# import xgboost as xgb
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

# # 学習、バリデーションデータ
# pred_xgb = np.zeros(len(train_df))
# pred_xgb_test = np.zeros(len(test_df))
# models_xgb = []

# # 入力データ
# X = X_train
# y = y_train

# # 評価履歴を保存する辞書
# evals_result_xgb = {}

# # パラメータ
# xgb_params = {
#     "objective": "binary:logistic",
#     "tree_method": "hist",
#     # "tree_method": "gpu_hist",
#     # "gpu_id": 0,
#     "eval_metric": "auc",
# }

# # クロスバリデーション
# # kf = KFold(n_splits=5, shuffle=True, random_state=42)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # for fold, (train_idx,valid_idx) in enumerate(kf.split(X)):
# for fold,(train_idx,valid_idx) in enumerate(skf.split(X,y)):

#     print("#"*25)
#     print(f"### Fold {fold+1}")
#     print("#"*25)

#     # foldごとの訓練、バリデーションデータ
#     X_train_kf = X.iloc[train_idx,:]
#     y_train_kf = y.iloc[train_idx]
#     X_valid_kf = X.iloc[valid_idx,:]
#     y_valid_kf = y.iloc[valid_idx]

#     # DMatrixに変換
#     dtrain = xgb.DMatrix(X_train_kf,label=y_train_kf)
#     dvalid = xgb.DMatrix(X_valid_kf,label=y_valid_kf)

#     # 学習
#     model_xgb = xgb.train(
#         xgb_params,
#         dtrain,
#         num_boost_round=2000,
#         evals=[(dtrain,"train"),(dvalid,"valid")],
#         early_stopping_rounds=100,
#         evals_result=evals_result_xgb,
#         verbose_eval=100,
#     )

#     # 各foldでのバリデーション予測
#     pred_xgb[valid_idx] = model_xgb.predict(
#         dvalid,
#         iteration_range=(0,model_xgb.best_iteration+1))

#     # モデルの追加
#     models_xgb.append(model_xgb)

#     # テストの予測
#     dtest = xgb.DMatrix(X_test)
#     pred_xgb_test = pred_xgb_test + model_xgb.predict(
#         dtest,
#         iteration_range=(0,model_xgb.best_iteration+1))

# # FOLD数で割る
# pred_xgb_test = pred_xgb_test/5

In [ ]:
# # 訓練データのスコア
# from sklearn.metrics import f1_score

# AUC_xgb = roc_auc_score(y,pred_xgb)
# F1_xgb = f1_score(y,np.round(pred_xgb,0))
# print(f"XGB: AUC score = {AUC_xgb}, F1 = {F1_xgb}")

In [ ]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_xgb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

In [ ]:
# # 学習曲線の描画
# from matplotlib.pyplot as plt
# epochs = len(evals_result_xgb['train']['auc'])
# x_axis = range(0, epochs)

# plt.figure()
# plt.plot(x_axis, evals_result_xgb['train']['auc'], label='Train')
# plt.plot(x_axis, evals_result_xgb['valid']['auc'], label='Validation')
# plt.xlabel('Iteration')
# plt.ylabel('AUC')
# plt.title('XGBoost AUC')
# plt.grid()
# plt.legend()
# plt.show()

### 【CatBoost】

In [ ]:
# #################################################
# ############ CatBoost ############################
# #################################################
# from catboost import Pool, train
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

# # 学習、バリデーションデータ
# pred_cb = np.zeros(len(train_df))
# pred_cb_test = np.zeros(len(test_df))
# models_cb = []

# # 入力データ
# X = X_train
# y = y_train

# # 履歴を保存
# cb_auc_valid = []

# cat_params = {
#     "loss_function": "Logloss",
#     "eval_metric": "AUC",
#     # "task_type": "CPU",
#     "task_type": "GPU",
#     # "devices": "0",
#     "iterations": 2000,
#     "verbose": 100,
# }

# # クロスバリデーション
# # kf = KFold(n_splits=5, shuffle=True, random_state=42)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # for fold, (train_idx,valid_idx) in enumerate(kf.split(X)):
# for fold,(train_idx,valid_idx) in enumerate(skf.split(X,y)):

#     print("#"*25)
#     print(f"### Fold {fold+1}")
#     print("#"*25)
    
#     X_train_kf = X.iloc[train_idx,:]
#     y_train_kf = y.iloc[train_idx]
#     X_valid_kf = X.iloc[valid_idx,:]
#     y_valid_kf = y.iloc[valid_idx]    

#     # object型をカテゴリ型に変換
#     for col in cat_col:
#         X_train_kf.loc[:,col] = X_train_kf.loc[:,col].astype("category")
#         X_valid_kf.loc[:,col] = X_valid_kf.loc[:,col].astype("category")

#     # データセット設定
#     train_pool = Pool(X_train_kf,y_train_kf,cat_features=cat_col)
#     valid_pool = Pool(X_valid_kf,y_valid_kf,cat_features=cat_col)

#     # 学習
#     model_cb = train(
#         params=cat_params,
#         dtrain=train_pool,
#         eval_set=valid_pool,
#         early_stopping_rounds=100)

#     # 各foldでのバリデーション予測
#     pred_cb[valid_idx] = model_cb.predict(X_valid_kf,
#         ntree_end=model_cb.best_iteration_,
#         prediction_type="Probability"
#     )[:,1]
    

#     # モデルを追加
#     models_cb.append(model_cb)

#     evals_result = model_cb.get_evals_result()
#     cb_auc_valid.append(evals_result["validation"]["AUC"])

#     # テストの予測
#     pred_cb_test = pred_cb_test + model_cb.predict(X_test,
#         ntree_end=model_cb.best_iteration_,
#         prediction_type="Probability"
#     )[:,1]

# # FOLD数で割る
# pred_cb_test = pred_cb_test/5

In [ ]:
# from sklearn.metrics import f1_score

# # スコア表示
# AUC_cb = roc_auc_score(y,pred_cb)
# F1_cb = f1_score(y,np.round(pred_cb,0))
# print(f"CB: AUC score = {AUC_cb}, F1 = {F1_cb}")

# # 学習履歴を一番短いfoldに揃える
# min_len = min(len(m) for m in cb_auc_valid)
# cb_auc_score = [m[:min_len] for m in cb_auc_valid]

# # foldごとの結果を平均する
# cb_auc_score = np.average(cb_auc_score,axis=0)

In [ ]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_cb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

In [ ]:
# import  matplotlib.pyplot as plt
# # 履歴の可視化
# plt.plot(cb_auc_score, label='Validation')
# plt.xlabel('Iteration')
# plt.ylabel('AUC')
# plt.grid()
# plt.legend()
# plt.title("CabBoost AUC")
# plt.show()

### 【Neural Net】

In [5]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# pytorch実装
import torch # Tensorの作成や操作
import torch.nn as nn # ニューラルネットワーク
import torch.nn.functional as F # 関数をメソッドとして提供
import torch.optim as optim # オプティマイザ
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

# GPUの使用状況確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# データフレーム読み込み
train_df = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")

# 設定
SEED = 42
CATEGORICAL = cat_col
NUMERICAL = num_col
TARGET = "y"
USE = CATEGORICAL + NUMERICAL
df_train = train_df.drop("id",axis=1)
df_test = test_df.drop("id",axis=1)

cuda:0


In [6]:
# 標準化 + ラベルエンコード
def preprocessing(df_train, df_test, cat_cols=CATEGORICAL, num_cols=NUMERICAL, target=TARGET):

    # 訓練データ + テストデータ
    df = pd.concat([df_train.drop(columns=target), df_test])
    y = df_train[target]
    train_len = len(df_train)
    
    # 欠損埋め
    df[cat_cols] = df[cat_cols].fillna('None')
    df[num_cols] = df[num_cols].fillna(0)

    train = df[:train_len]
    test = df[train_len:]

    # 標準化
    scaler = StandardScaler()

    # フィッティング
    # scaler.fit(df[num_cols])
    scaler.fit(train[num_cols])

    # 適用
    train[num_cols] = scaler.transform(train[num_cols])
    test[num_cols] = scaler.transform(test[num_cols])
    df = pd.concat([train, test])
    
    # ラベルエンコーダ
    for col in df.columns:
        if col in cat_cols:
            df[col] = LabelEncoder().fit_transform(df[col])
            df[col]= df[col].astype('category')
            
    return pd.concat([df.iloc[:train_len], y], axis=1), df.iloc[train_len:]

In [7]:
# データセット関数
class CustomDataset(Dataset):

    # オブジェクト定義
    def __init__(self, df, target, cat_cols=CATEGORICAL):
        self.df_cat = df[cat_cols]
        self.df_num = df.drop(cat_cols, axis=1)
        self.X_cats = self.df_cat.values.astype(np.int64)
        self.X_nums = self.df_num.values.astype(np.float32)
        self.target = target.values.astype(np.int64)

    # データセットのサイズを返す
    def __len__(self):
        return len(self.target)

    # 指定したインデックスのデータとラベルを返す
    def __getitem__(self, idx):
        return [self.X_cats[idx], self.X_nums[idx], self.target[idx]]

In [8]:
# NNモデル作成
class NN_Model(nn.Module):

    # ネットワーク構造の定義
    def __init__(self, embedding_sizes, n_num):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories, size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings)
        self.n_emb, self.n_num = n_emb, n_num
        self.lin1 = nn.Linear(self.n_emb + self.n_num, 100)
        self.lin2 = nn.Linear(100, 70)
        self.lin3 = nn.Linear(70, 2)
        self.bn1 = nn.BatchNorm1d(self.n_num)
        self.bn2 = nn.BatchNorm1d(100)
        self.bn3 = nn.BatchNorm1d(70)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
 
    # 順伝播
    def forward(self,x_cat,x_num):
        x = [e(x_cat[:, i]) for i, e in enumerate(self.embeddings)]
        x = torch.cat(x, dim=1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_num)
        x = torch.cat([x, x2], dim=1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [9]:
# 前処理の実施
df_train, df_test = preprocessing(df_train, df_test)

# ラベルエンコード済みカテゴリ変数の埋め込み
# 各カテゴリ列の変数の種類
cat_sizes = [len(df_train[col].cat.categories) for col in CATEGORICAL]

# (入力サイズ, 50と割る2の小さい方)でエンコード
emb_sizes = [(size, min(50, (size+1)//2)) for size in cat_sizes]

/tmp/ipykernel_36/1479464330.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[num_cols] = scaler.transform(train[num_cols])
/tmp/ipykernel_36/1479464330.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[num_cols] = scaler.transform(test[num_cols])


In [ ]:
# 記録用
hist = {
    'train_loss': [], 'train_auc': [],
    'val_loss': [], 'val_auc': []
}

# パラメータ
bs = 64 # バッチサイズ
EPOCHS = 10 # エポック
FOLDS = 5 # FOLD数
LR=1e-3 # 学習率

# stratified KFoldの宣言
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

fold_results = []

test_results = []

# SKFによるデータ分割
for fold, (train_idx, val_idx) in enumerate(skf.split(df_train.drop(columns=TARGET), df_train[TARGET])):
    
    print(f"\n========== Fold {fold+1} ==========")

    # 学習データ
    X_train = df_train.drop(columns=TARGET).iloc[train_idx] 
    y_train = df_train[TARGET].iloc[train_idx]

    # バリデーションデータ
    X_val = df_train.drop(columns=TARGET).iloc[val_idx]
    y_val = df_train[TARGET].iloc[val_idx]

    # Datasetの作成
    train_dataset = CustomDataset(X_train, y_train)
    val_dataset = CustomDataset(X_val, y_val)
    
    # DataLoaderの作成
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False)

    # モデル構築
    model = NN_Model(emb_sizes, len(NUMERICAL)).to(device)

    # 最適化設定
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    # 損失関数
    criterion = nn.CrossEntropyLoss()

    hist = {"train_auc": [], "val_auc": []}

    # 学習・予測エポックのループ
    for epoch in range(EPOCHS):

        # 開始時間
        start_time = time.time()

        # 学習モード
        model.train()

        # ラベル、予測値の保存場所
        y_true_train, y_pred_train = [], []

        # プログレスバー
        train_iter = tqdm(train_loader, desc=f"<Train> Epoch {epoch+1}", leave=False)
        
        for cat_data, num_data, target in train_iter:

            # DataLoaderから取り出した、カテゴリ、数値、ターゲット
            cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)

            # パラメータの勾配を初期化
            optimizer.zero_grad()

            # 予測値の算出
            output = model(cat_data, num_data)

            # ラベルと予測値とのロス計算
            loss = criterion(output, target)

            # 各パラメータの勾配を算出
            loss.backward()

            # パラメータ更新
            optimizer.step()

            # ソフトマックスの分類結果を格納
            probs = torch.softmax(output, dim=1)[:, 1].detach().cpu().numpy()
            y_pred_train.extend(probs)

            # ラベルの格納
            y_true_train.extend(target.cpu().numpy())

            # プログレスバーの後ろにロス値を表示
            train_iter.set_postfix(loss=loss.item())

        # histに残すAUCスコア
        train_auc = roc_auc_score(y_true_train, y_pred_train)

        # 評価モード
        model.eval()

        # ラベル、予測値の保存場所        
        y_true_val, y_pred_val = [], []

        # プログレスバー
        val_iter = tqdm(val_loader, desc=f"<Val> Epoch {epoch+1}", leave=False)

        # 勾配を更新しない
        with torch.no_grad():
            
            for cat_data, num_data, target in val_iter:
    
                # DataLoaderから取り出した、カテゴリ、数値、ターゲット
                cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)

                # 予測値の算出
                output = model(cat_data, num_data)

                # ソフトマックスの分類結果を格納
                probs = torch.softmax(output, dim=1)[:, 1].cpu().numpy()
                y_pred_val.extend(probs)

                # ラベルの格納
                y_true_val.extend(target.cpu().numpy())

                # プログレスバーの後ろにロス値を表示
                val_iter.set_postfix(loss=criterion(output, target).item())

        # histに残すAUCスコア        
        val_auc = roc_auc_score(y_true_val, y_pred_val)

        # 差分時刻
        elapsed = time.time() - start_time

        # 履歴追加
        hist["train_auc"].append(train_auc)
        hist["val_auc"].append(val_auc)

        # 進捗
        print(f"Epoch {epoch+1}/{EPOCHS} - TrainAUC: {train_auc:.4f} | ValAUC: {val_auc:.4f} | Time: {elapsed:.1f}s")

    # foldごとに保存
    torch.save(model.state_dict(), f"model_fold{fold+1}.pth")
    
    # ヒストグラムの更新
    fold_results.append(hist)

    # foldごとにテストデータ計算
    model.eval()
    with torch.no_grad():
        X_test_cat = torch.from_numpy(df_test[CATEGORICAL].values.astype(np.int64)).to(device)
        X_test_num = torch.from_numpy(df_test[NUMERICAL].values.astype(np.float32)).to(device)

        # 予測
        preds = torch.softmax(model(X_test_cat, X_test_num),dim=1)[:,1].cpu().numpy()
        # preds = torch.softmax(model(X_test_cat, X_test_num).squeeze()).cpu().numpy()
        test_results.append(preds)
    
# shape = (n_folds, n_test_samples) → 平均化
test_results = np.mean(test_results, axis=0)        


========== Fold 1 ==========


Epoch 1/10 - TrainAUC: 0.9187 | ValAUC: 0.9503 | Time: 62.5s


<Val> Epoch 2:  62%|██████▏   | 1444/2344 [00:04<00:02, 330.46it/s, loss=0.237]   

In [ ]:
len(test_results)

In [ ]:
# 提出データ作成
sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

sample_submission['y'] = test_results
sample_submission.to_csv('submission.csv', index=False)
print('Submission file saved.')

In [ ]:
# # ======================
# # FoldごとのAUCをプロット
# # ======================
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,5))
# for i, hist in enumerate(fold_results):
#     plt.plot(hist["val_auc"], label=f"Fold {i+1} Val AUC")
# plt.xlabel("Epoch")
# plt.ylabel("AUC")
# plt.legend()
# plt.title("Validation AUC per Fold")
# plt.show() 

In [ ]:
# # 提出データ作成
# sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

# sample_submission['y'] = pred_lgb_test
# sample_submission.to_csv('submission.csv', index=False)
# print('Submission file saved.')

In [ ]:
# # 検証データの誤差の可視化
# train_loss = hist['train_loss']
# val_loss = hist['val_loss']

# fig = plt.figure()
# plt.rc('font', family='serif')
# plt.plot(range(len(train_loss)), train_loss, linewidth=1, label='train_loss')
# plt.plot(range(len(val_loss)), val_loss, color='red', linewidth=1, label='val_loss')
# plt.xlabel('epochs')
# plt.ylabel('loss')
# plt.legend()
# plt.show()

In [ ]:
# print(f'validation data best acc : {best_acc}')

# # 検証データの精度の可視化
# train_acc = hist['train_accuracy']
# val_acc = hist['val_accuracy']

# fig = plt.figure()
# plt.rc('font', family='serif')
# plt.plot(range(len(train_acc)), train_acc, linewidth=1, label='train_acc')
# plt.plot(range(len(val_acc)), val_acc, color='red', linewidth=1, label='val_acc')
# plt.xlabel('epochs')
# plt.ylabel('accuracy')
# plt.legend()
# plt.show()

In [ ]:
# # 記録用
# hist = {
#     'train_loss': [], 'train_auc': [],
#     'val_loss': [], 'val_auc': []
# }

# # パラメータ
# bs = 64 # バッチサイズ
# EPOCHS = 5 # エポック
# FOLDS = 5 # FOLD数
# LR=1e-3 # 学習率

# # stratified KFoldの宣言
# skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

# fold_results = []

# for fold, (train_idx, val_idx) in enumerate(skf.split(df_train.drop(columns=TARGET), df_train[TARGET])):
    
#     print(f"\n========== Fold {fold+1} ==========")

#     # データ分割
#     X_train, X_val = df_train.drop(columns=TARGET).iloc[train_idx], df_train.drop(columns=TARGET).iloc[val_idx]
#     y_train, y_val = df_train[TARGET].iloc[train_idx], df_train[TARGET].iloc[val_idx]

#     # Dataset / DataLoader
#     train_dataset = CustomDataset(X_train, y_train)
#     val_dataset = CustomDataset(X_val, y_val)
#     train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False)

#     # モデル・最適化・損失
#     model = NN_Model(emb_sizes, len(NUMERICAL)).to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=LR)
#     criterion = nn.CrossEntropyLoss()

#     hist = {"train_auc": [], "val_auc": []}

#     for epoch in range(EPOCHS):
#         start_time = time.time()

#         # ===== 学習 =====
#         model.train()
#         y_true_train, y_pred_train = [], []
#         train_iter = tqdm(train_loader, desc=f"<Train> Epoch {epoch+1}", leave=False)
#         for cat_data, num_data, target in train_iter:
#             cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)

#             optimizer.zero_grad()
#             output = model(cat_data, num_data)
#             loss = criterion(output, target)
#             loss.backward()
#             optimizer.step()

#             probs = torch.softmax(output, dim=1)[:, 1].detach().cpu().numpy()
#             y_pred_train.extend(probs)
#             y_true_train.extend(target.cpu().numpy())
#             train_iter.set_postfix(loss=loss.item())

#         train_auc = roc_auc_score(y_true_train, y_pred_train)

#         # ===== 検証 =====
#         model.eval()
#         y_true_val, y_pred_val = [], []
#         val_iter = tqdm(val_loader, desc=f"<Val> Epoch {epoch+1}", leave=False)
#         with torch.no_grad():
#             for cat_data, num_data, target in val_iter:
#                 cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)
#                 output = model(cat_data, num_data)
#                 probs = torch.softmax(output, dim=1)[:, 1].cpu().numpy()
#                 y_pred_val.extend(probs)
#                 y_true_val.extend(target.cpu().numpy())
#                 val_iter.set_postfix(loss=criterion(output, target).item())

#         val_auc = roc_auc_score(y_true_val, y_pred_val)
#         elapsed = time.time() - start_time

#         hist["train_auc"].append(train_auc)
#         hist["val_auc"].append(val_auc)

#         print(f"Epoch {epoch+1}/{EPOCHS} - TrainAUC: {train_auc:.4f} | ValAUC: {val_auc:.4f} | Time: {elapsed:.1f}s")

#     fold_results.append(hist)

#     # テストデータ

# # ======================
# # FoldごとのAUCをプロット
# # ======================
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,5))
# for i, hist in enumerate(fold_results):
#     plt.plot(hist["val_auc"], label=f"Fold {i+1} Val AUC")
# plt.xlabel("Epoch")
# plt.ylabel("AUC")
# plt.legend()
# plt.title("Validation AUC per Fold")
# plt.show() 